In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [70]:
df = pd.read_csv('data/samples_updated.csv')
df

,filename,subject,group,race,age,x0,y0,x1,y1,x2,...,x63,y63,x64,y64,x65,y65,x66,y66,x67,y67
0,E:\Lurie Research\final cohort3\CCHS\Caucasian...,0,CCHS,Caucasian,0-1,1725,1074,1721,1172,1723,...,2144,1352,2264,1420,2144,1505,2090,1511,2038,1504
1,E:\Lurie Research\final cohort3\CCHS\Caucasian...,0,CCHS,Caucasian,0-1,1576,1083,1568,1183,1561,...,1988,1421,2096,1464,1984,1491,1935,1493,1887,1485
2,E:\Lurie Research\final cohort3\CCHS\Caucasian...,0,CCHS,Caucasian,0-1,1384,954,1365,1043,1348,...,1701,1268,1820,1303,1695,1297,1658,1295,1620,1288
3,E:\Lurie Research\final cohort3\CCHS\Caucasian...,0,CCHS,Caucasian,0-1,1396,1095,1383,1188,1369,...,1737,1435,1858,1464,1732,1473,1688,1474,1646,1468
4,E:\Lurie Research\final cohort3\CCHS\Caucasian...,0,CCHS,Caucasian,3-Feb,1430,1397,1470,1509,1510,...,2065,1469,2146,1542,2071,1582,2027,1592,1981,1597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,E:\Lurie Research\final cohort3\CONTROL\Other\...,1828,CONTROL,Other,26-35,179,119,181,142,186,...,268,208,281,210,268,214,259,215,250,214
2884,E:\Lurie Research\final cohort3\CONTROL\Other\...,1829,CONTROL,Other,26-35,95,289,99,326,106,...,255,413,288,416,255,429,241,431,225,428
2885,E:\Lurie Research\final cohort3\CONTROL\Other\...,1830,CONTROL,Other,26-35,190,115,188,131,187,...,236,171,253,174,236,179,229,179,223,177
2886,E:\Lurie Research\final cohort3\CONTROL\Other\...,1831,CONTROL,Other,26-35,310,261,311,325,318,...,574,473,606,477,573,475,550,478,527,475


In [5]:
duplicate_subject = df[df.duplicated('subject', keep=False)]
duplicate_subject

,filename,subject,group,race,age,x0,y0,x1,y1,x2,...,x63,y63,x64,y64,x65,y65,x66,y66,x67,y67
1,E:\Lurie Research\final cohort3\CONTROL\Africa...,1453,CONTROL,African American,0-1,61,115,63,131,67,...,130,175,142,179,130,177,126,177,122,176
21,E:\Lurie Research\final cohort3\CONTROL\Africa...,1473,CONTROL,African American,0-1,49,130,52,147,55,...,139,167,149,177,139,183,131,184,123,183
30,E:\Lurie Research\final cohort3\CONTROL\Africa...,1482,CONTROL,African American,0-1,62,116,65,131,69,...,133,170,144,171,133,170,129,171,125,170
33,E:\Lurie Research\final cohort3\CONTROL\Africa...,1485,CONTROL,African American,0-1,47,127,50,143,53,...,134,171,150,172,134,172,129,172,124,172
34,E:\Lurie Research\final cohort3\CONTROL\Africa...,1486,CONTROL,African American,2-3,46,116,48,136,52,...,136,173,146,179,137,181,133,181,127,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,E:\Lurie Research\final cohort3\CCHS\Other\18-...,98,CCHS,Other,18-25,1473,801,1486,891,1502,...,1889,1017,1961,1040,1889,1022,1854,1026,1821,1026
2885,E:\Lurie Research\final cohort3\CCHS\Other\18-...,98,CCHS,Other,18-25,1496,1134,1506,1223,1522,...,1908,1352,1980,1374,1909,1352,1874,1356,1842,1357
2886,E:\Lurie Research\final cohort3\CCHS\Other\26-...,108,CCHS,Other,26-35,131,207,134,237,137,...,272,301,299,297,272,304,260,306,248,305
2887,E:\Lurie Research\final cohort3\CCHS\Other\26-...,108,CCHS,Other,26-35,1130,491,1130,555,1131,...,1405,680,1471,686,1404,680,1382,681,1359,679


In [69]:
# get row #s of duplicated subjects
duplicated_rows = {}
rows = []
# for subject in duplicate_subject['subject']:
#     
for idx, row in duplicate_subject.iterrows():
    subject = row['subject']
    if subject not in duplicated_rows.keys(): # add key
        duplicated_rows[subject] = [idx]
    else:
        duplicated_rows[subject].append(idx)
    rows.append(idx)
len(cchs_rows)

728

In [13]:
# load data
X = torch.from_numpy(np.load('data/stratify_people/samples.npy'))
y = torch.from_numpy(np.load('data/stratify_people/labels.npy'))

df_X = pd.DataFrame(X)
df_X = df_X.rename(columns={0: 'subject'})
df_y = pd.DataFrame(y)
df_y = df_y.rename(columns={0: 'label'})
df_X

,subject,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,tensor(1452.),tensor(1.),tensor(0.),tensor(0.),tensor(0.2931),tensor(0.0388),tensor(0.4095),tensor(0.0776),tensor(0.5194),tensor(0.1207),...,tensor(0.5647),tensor(0.6099),tensor(0.6573),tensor(0.6099),tensor(0.5711),tensor(0.6250),tensor(0.5431),tensor(0.6336),tensor(0.5129),tensor(0.6358)
1,tensor(1453.),tensor(1.),tensor(0.),tensor(0.),tensor(0.1408),tensor(0.0141),tensor(0.2535),tensor(0.0423),tensor(0.3662),tensor(0.0704),...,tensor(0.4859),tensor(0.5634),tensor(0.5704),tensor(0.5915),tensor(0.4859),tensor(0.5775),tensor(0.4577),tensor(0.5775),tensor(0.4296),tensor(0.5704)
2,tensor(1454.),tensor(1.),tensor(0.),tensor(0.),tensor(0.1631),tensor(0.0043),tensor(0.2983),tensor(0.0107),tensor(0.4335),tensor(0.0279),...,tensor(0.5815),tensor(0.6202),tensor(0.6395),tensor(0.6567),tensor(0.5794),tensor(0.6588),tensor(0.5429),tensor(0.6631),tensor(0.5064),tensor(0.6567)
3,tensor(1455.),tensor(1.),tensor(0.),tensor(0.1272),tensor(0.1422),tensor(0.0819),tensor(0.2349),tensor(0.0431),tensor(0.3319),tensor(0.0129),...,tensor(0.3341),tensor(0.7284),tensor(0.4332),tensor(0.8470),tensor(0.3125),tensor(0.7608),tensor(0.2759),tensor(0.7241),tensor(0.2500),tensor(0.6832)
4,tensor(1456.),tensor(1.),tensor(0.),tensor(0.0244),tensor(0.0325),tensor(0.0122),tensor(0.1667),tensor(0.0041),tensor(0.3049),tensor(0.),...,tensor(0.5041),tensor(0.6951),tensor(0.5650),tensor(0.7154),tensor(0.4959),tensor(0.6748),tensor(0.4634),tensor(0.6667),tensor(0.4309),tensor(0.6545)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,tensor(98.),tensor(0.),tensor(7.),tensor(0.),tensor(0.3119),tensor(0.0178),tensor(0.4350),tensor(0.0397),tensor(0.5540),tensor(0.0643),...,tensor(0.5691),tensor(0.6074),tensor(0.6676),tensor(0.6389),tensor(0.5691),tensor(0.6142),tensor(0.5212),tensor(0.6197),tensor(0.4761),tensor(0.6197)
2885,tensor(98.),tensor(0.),tensor(7.),tensor(0.),tensor(0.3182),tensor(0.0138),tensor(0.4408),tensor(0.0358),tensor(0.5606),tensor(0.0620),...,tensor(0.5675),tensor(0.6185),tensor(0.6667),tensor(0.6488),tensor(0.5689),tensor(0.6185),tensor(0.5207),tensor(0.6240),tensor(0.4766),tensor(0.6253)
2886,tensor(108.),tensor(0.),tensor(8.),tensor(0.),tensor(0.2407),tensor(0.0124),tensor(0.3651),tensor(0.0249),tensor(0.4855),tensor(0.0456),...,tensor(0.5851),tensor(0.6307),tensor(0.6971),tensor(0.6141),tensor(0.5851),tensor(0.6432),tensor(0.5353),tensor(0.6515),tensor(0.4855),tensor(0.6473)
2887,tensor(108.),tensor(0.),tensor(8.),tensor(0.),tensor(0.3092),tensor(0.),tensor(0.4313),tensor(0.0019),tensor(0.5496),tensor(0.0153),...,tensor(0.5248),tensor(0.6698),tensor(0.6508),tensor(0.6813),tensor(0.5229),tensor(0.6698),tensor(0.4809),tensor(0.6718),tensor(0.4370),tensor(0.6679)


In [60]:
cchs_rows = {} # idx:subject
for idx, row in df_X.iterrows():
    subject = int(row['subject'].item())
    label = int(df_y.iloc[idx]['label'].item())
    if label == 1: # cchs
        cchs_rows[idx] = subject
len(cchs_rows)

728

In [66]:
buckets_cchs = [[] for i in range(0, 10)] # 10 buckets for 10 fold; buckets contain row numbers
# deal with duplicate cchs first
for subject, rows in duplicated_rows.items():
    if subject in cchs_rows.values(): # duplicate subject is a cchs subject
        smallest_bucket = buckets_cchs.index(sorted(buckets_cchs, key=len)[0]) # index of smallest bucket
        for idx in rows:
            buckets_cchs[smallest_bucket].append(idx)
[len(i) for i in buckets_cchs]

[66, 70, 71, 70, 66, 66, 75, 65, 79, 73]

In [64]:
# add non-duplicate cchs
for subject in cchs_rows.values():
    if subject not in duplicated_rows.keys(): # duplicate subject is a cchs subject
        smallest_bucket = buckets_cchs.index(sorted(buckets_cchs, key=len)[0]) # index of smallest bucket
        for idx in rows:
            buckets_cchs[smallest_bucket].append(idx)
[len(i) for i in buckets_cchs]

[219, 219, 219, 219, 219, 219, 219, 219, 216, 216]

# old

In [32]:
df_X = original_df_X.drop(index=rows)
df_y = original_df_y.drop(index=rows)
df_X 

,subject,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,tensor(1452.),tensor(1.),tensor(0.),tensor(0.),tensor(0.2931),tensor(0.0388),tensor(0.4095),tensor(0.0776),tensor(0.5194),tensor(0.1207),...,tensor(0.5647),tensor(0.6099),tensor(0.6573),tensor(0.6099),tensor(0.5711),tensor(0.6250),tensor(0.5431),tensor(0.6336),tensor(0.5129),tensor(0.6358)
1,tensor(1453.),tensor(1.),tensor(0.),tensor(0.),tensor(0.1408),tensor(0.0141),tensor(0.2535),tensor(0.0423),tensor(0.3662),tensor(0.0704),...,tensor(0.4859),tensor(0.5634),tensor(0.5704),tensor(0.5915),tensor(0.4859),tensor(0.5775),tensor(0.4577),tensor(0.5775),tensor(0.4296),tensor(0.5704)
2,tensor(1454.),tensor(1.),tensor(0.),tensor(0.),tensor(0.1631),tensor(0.0043),tensor(0.2983),tensor(0.0107),tensor(0.4335),tensor(0.0279),...,tensor(0.5815),tensor(0.6202),tensor(0.6395),tensor(0.6567),tensor(0.5794),tensor(0.6588),tensor(0.5429),tensor(0.6631),tensor(0.5064),tensor(0.6567)
3,tensor(1455.),tensor(1.),tensor(0.),tensor(0.1272),tensor(0.1422),tensor(0.0819),tensor(0.2349),tensor(0.0431),tensor(0.3319),tensor(0.0129),...,tensor(0.3341),tensor(0.7284),tensor(0.4332),tensor(0.8470),tensor(0.3125),tensor(0.7608),tensor(0.2759),tensor(0.7241),tensor(0.2500),tensor(0.6832)
4,tensor(1456.),tensor(1.),tensor(0.),tensor(0.0244),tensor(0.0325),tensor(0.0122),tensor(0.1667),tensor(0.0041),tensor(0.3049),tensor(0.),...,tensor(0.5041),tensor(0.6951),tensor(0.5650),tensor(0.7154),tensor(0.4959),tensor(0.6748),tensor(0.4634),tensor(0.6667),tensor(0.4309),tensor(0.6545)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2816,tensor(104.),tensor(0.),tensor(2.),tensor(0.),tensor(0.1031),tensor(0.0185),tensor(0.2354),tensor(0.0415),tensor(0.3585),tensor(0.0677),...,tensor(0.5231),tensor(0.6292),tensor(0.5954),tensor(0.6431),tensor(0.5246),tensor(0.6369),tensor(0.4938),tensor(0.6385),tensor(0.4631),tensor(0.6354)
2820,tensor(105.),tensor(0.),tensor(2.),tensor(0.),tensor(0.3553),tensor(0.0121),tensor(0.4693),tensor(0.0274),tensor(0.5789),tensor(0.0471),...,tensor(0.5680),tensor(0.5691),tensor(0.6776),tensor(0.6261),tensor(0.5658),tensor(0.6283),tensor(0.5121),tensor(0.6305),tensor(0.4572),tensor(0.6305)
2826,tensor(106.),tensor(0.),tensor(3.),tensor(0.),tensor(0.1762),tensor(0.0081),tensor(0.3126),tensor(0.0234),tensor(0.4440),tensor(0.0560),...,tensor(0.5703),tensor(0.6395),tensor(0.6650),tensor(0.6324),tensor(0.5692),tensor(0.6415),tensor(0.5285),tensor(0.6497),tensor(0.4919),tensor(0.6466)
2852,tensor(107.),tensor(0.),tensor(4.),tensor(0.),tensor(0.1278),tensor(0.0142),tensor(0.2656),tensor(0.0355),tensor(0.3949),tensor(0.0597),...,tensor(0.5312),tensor(0.6420),tensor(0.6250),tensor(0.6520),tensor(0.5312),tensor(0.6477),tensor(0.4957),tensor(0.6506),tensor(0.4574),tensor(0.6491)


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.10, stratify=df_y)
X_train

,subject,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
693,tensor(659.),tensor(-1.),tensor(2.),tensor(0.),tensor(0.1352),tensor(0.0143),tensor(0.2541),tensor(0.0307),tensor(0.3709),tensor(0.0553),...,tensor(0.5533),tensor(0.5492),tensor(0.6537),tensor(0.5471),tensor(0.5553),tensor(0.5594),tensor(0.5307),tensor(0.5635),tensor(0.5000),tensor(0.5594)
1570,tensor(1340.),tensor(-1.),tensor(6.),tensor(0.1091),tensor(0.0273),tensor(0.0636),tensor(0.1455),tensor(0.0364),tensor(0.2727),tensor(0.0091),...,tensor(0.4545),tensor(0.6455),tensor(0.5727),tensor(0.6909),tensor(0.4364),tensor(0.7091),tensor(0.3727),tensor(0.7000),tensor(0.3273),tensor(0.6636)
108,tensor(1555.),tensor(1.),tensor(3.),tensor(0.),tensor(0.0500),tensor(0.0175),tensor(0.1825),tensor(0.0475),tensor(0.3075),tensor(0.0800),...,tensor(0.5025),tensor(0.6175),tensor(0.5850),tensor(0.6325),tensor(0.5000),tensor(0.6275),tensor(0.4500),tensor(0.6350),tensor(0.4050),tensor(0.6275)
886,tensor(797.),tensor(-1.),tensor(3.),tensor(0.),tensor(0.1585),tensor(0.0109),tensor(0.2896),tensor(0.0437),tensor(0.4098),tensor(0.0820),...,tensor(0.5956),tensor(0.6011),tensor(0.6667),tensor(0.6120),tensor(0.5956),tensor(0.6011),tensor(0.5628),tensor(0.6066),tensor(0.5301),tensor(0.6066)
33,tensor(1485.),tensor(1.),tensor(0.),tensor(0.),tensor(0.2092),tensor(0.0196),tensor(0.3137),tensor(0.0392),tensor(0.4118),tensor(0.0588),...,tensor(0.5686),tensor(0.4967),tensor(0.6732),tensor(0.5033),tensor(0.5686),tensor(0.5033),tensor(0.5359),tensor(0.5033),tensor(0.5033),tensor(0.5033)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,tensor(1808.),tensor(0.),tensor(5.),tensor(0.),tensor(0.2235),tensor(0.0118),tensor(0.3529),tensor(0.0353),tensor(0.4824),tensor(0.0706),...,tensor(0.6235),tensor(0.6529),tensor(0.7176),tensor(0.6588),tensor(0.6235),tensor(0.6529),tensor(0.5706),tensor(0.6647),tensor(0.5235),tensor(0.6588)
106,tensor(1553.),tensor(1.),tensor(3.),tensor(0.),tensor(0.4858),tensor(0.0388),tensor(0.5917),tensor(0.0904),tensor(0.7003),tensor(0.1525),...,tensor(0.6744),tensor(0.5711),tensor(0.7855),tensor(0.5995),tensor(0.6873),tensor(0.6718),tensor(0.6382),tensor(0.6925),tensor(0.5840),tensor(0.7028)
319,tensor(342.),tensor(-1.),tensor(0.),tensor(0.),tensor(0.2443),tensor(0.0382),tensor(0.3511),tensor(0.0763),tensor(0.4580),tensor(0.1221),...,tensor(0.5420),tensor(0.5725),tensor(0.6489),tensor(0.5573),tensor(0.5496),tensor(0.5878),tensor(0.5267),tensor(0.5954),tensor(0.4962),tensor(0.5878)
241,tensor(254.),tensor(-1.),tensor(0.),tensor(0.),tensor(0.1293),tensor(0.0272),tensor(0.2517),tensor(0.0544),tensor(0.3741),tensor(0.0884),...,tensor(0.5238),tensor(0.4762),tensor(0.6531),tensor(0.4966),tensor(0.5374),tensor(0.5918),tensor(0.4898),tensor(0.6054),tensor(0.4490),tensor(0.5986)


In [20]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1530, 139)
(171, 139)
(1530, 1)
(171, 1)


In [12]:
# add to training set first
for idx, row in X_train.iterrows():
    subject = int(row['subject'].item())
    if subject in duplicated_rows.keys():
        for row in duplicated_rows[subject]:
            X_train = X_train.append(original_df_X.iloc[row])
            y_train = y_train.append(original_df_y.iloc[row])
X_train

,subject,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
62,tensor(1515.),tensor(1.),tensor(1.),tensor(0.),tensor(0.3909),tensor(0.0455),tensor(0.5182),tensor(0.1000),tensor(0.6273),tensor(0.1727),...,tensor(0.7182),tensor(0.6364),tensor(0.7909),tensor(0.6364),tensor(0.7182),tensor(0.6818),tensor(0.6818),tensor(0.7000),tensor(0.6364),tensor(0.7091)
1994,tensor(1727.),tensor(0.),tensor(3.),tensor(0.),tensor(0.2258),tensor(0.0036),tensor(0.3333),tensor(0.0143),tensor(0.4409),tensor(0.0323),...,tensor(0.4731),tensor(0.5556),tensor(0.6093),tensor(0.5878),tensor(0.4767),tensor(0.6129),tensor(0.4265),tensor(0.6201),tensor(0.3799),tensor(0.6201)
242,tensor(255.),tensor(-1.),tensor(0.),tensor(0.),tensor(0.2249),tensor(0.0271),tensor(0.3306),tensor(0.0515),tensor(0.4363),tensor(0.0759),...,tensor(0.5610),tensor(0.4797),tensor(0.6721),tensor(0.5014),tensor(0.5610),tensor(0.5339),tensor(0.5230),tensor(0.5366),tensor(0.4824),tensor(0.5339)
261,tensor(274.),tensor(-1.),tensor(0.),tensor(0.),tensor(0.4309),tensor(0.0461),tensor(0.5395),tensor(0.0954),tensor(0.6447),tensor(0.1447),...,tensor(0.6151),tensor(0.6184),tensor(0.7072),tensor(0.6283),tensor(0.6250),tensor(0.6711),tensor(0.5954),tensor(0.6875),tensor(0.5658),tensor(0.6941)
612,tensor(619.),tensor(-1.),tensor(1.),tensor(0.),tensor(0.3708),tensor(0.0497),tensor(0.4811),tensor(0.1070),tensor(0.5795),tensor(0.1697),...,tensor(0.6205),tensor(0.4811),tensor(0.7178),tensor(0.5038),tensor(0.6346),tensor(0.5611),tensor(0.5978),tensor(0.5762),tensor(0.5546),tensor(0.5838)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,tensor(62.),tensor(-1.),tensor(6.),tensor(0.0015),tensor(0.1944),tensor(0.),tensor(0.3175),tensor(0.0074),tensor(0.4377),tensor(0.0193),...,tensor(0.5697),tensor(0.6424),tensor(0.6914),tensor(0.6513),tensor(0.5653),tensor(0.6365),tensor(0.5104),tensor(0.6395),tensor(0.4555),tensor(0.6276)
2660,tensor(62.),tensor(-1.),tensor(6.),tensor(0.),tensor(0.2183),tensor(0.0032),tensor(0.3493),tensor(0.0181),tensor(0.4814),tensor(0.0394),...,tensor(0.5612),tensor(0.6603),tensor(0.6848),tensor(0.6773),tensor(0.5602),tensor(0.6635),tensor(0.5016),tensor(0.6667),tensor(0.4452),tensor(0.6581)
2661,tensor(62.),tensor(-1.),tensor(6.),tensor(0.),tensor(0.2288),tensor(0.),tensor(0.3584),tensor(0.0142),tensor(0.4869),tensor(0.0381),...,tensor(0.5588),tensor(0.6699),tensor(0.6786),tensor(0.6863),tensor(0.5577),tensor(0.6732),tensor(0.5011),tensor(0.6765),tensor(0.4488),tensor(0.6688)
2727,tensor(62.),tensor(-1.),tensor(7.),tensor(0.),tensor(0.2500),tensor(0.0054),tensor(0.3884),tensor(0.0139),tensor(0.5236),tensor(0.0386),...,tensor(0.6234),tensor(0.6427),tensor(0.7189),tensor(0.6599),tensor(0.6255),tensor(0.6266),tensor(0.5773),tensor(0.6288),tensor(0.5258),tensor(0.6245)


In [13]:
# add to test set first
for idx, row in X_test.iterrows():
    subject = int(row['subject'].item())
    if subject in duplicated_rows.keys():
        for row in duplicated_rows[subject]:
            X_test = X_test.append(original_df_X.iloc[row])
            y_test = y_test.append(original_df_y.iloc[row])
X_test

,subject,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
1366,tensor(1192.),tensor(-1.),tensor(5.),tensor(0.),tensor(0.2397),tensor(0.0248),tensor(0.3636),tensor(0.0661),tensor(0.4793),tensor(0.1157),...,tensor(0.5620),tensor(0.5537),tensor(0.7025),tensor(0.5455),tensor(0.5702),tensor(0.6529),tensor(0.5124),tensor(0.6694),tensor(0.4545),tensor(0.6694)
2223,tensor(0.),tensor(-1.),tensor(0.),tensor(0.0045),tensor(0.1960),tensor(0.),tensor(0.3057),tensor(0.0022),tensor(0.4177),tensor(0.0101),...,tensor(0.4737),tensor(0.5073),tensor(0.6081),tensor(0.5834),tensor(0.4737),tensor(0.6786),tensor(0.4132),tensor(0.6853),tensor(0.3550),tensor(0.6775)
2001,tensor(1735.),tensor(0.),tensor(3.),tensor(0.),tensor(0.0250),tensor(0.0200),tensor(0.1600),tensor(0.0400),tensor(0.2850),tensor(0.0650),...,tensor(0.5250),tensor(0.5600),tensor(0.6750),tensor(0.5300),tensor(0.5250),tensor(0.6200),tensor(0.4650),tensor(0.6300),tensor(0.4050),tensor(0.6150)
1769,tensor(1436.),tensor(-1.),tensor(7.),tensor(0.0194),tensor(0.1262),tensor(0.),tensor(0.2524),tensor(0.),tensor(0.3689),tensor(0.0194),...,tensor(0.3981),tensor(0.6019),tensor(0.5534),tensor(0.6311),tensor(0.3883),tensor(0.6699),tensor(0.3301),tensor(0.6602),tensor(0.2718),tensor(0.6408)
1877,tensor(1633.),tensor(0.),tensor(1.),tensor(0.),tensor(0.1014),tensor(0.0290),tensor(0.2319),tensor(0.0652),tensor(0.3551),tensor(0.0942),...,tensor(0.6087),tensor(0.5725),tensor(0.6739),tensor(0.5797),tensor(0.6087),tensor(0.5870),tensor(0.5797),tensor(0.5942),tensor(0.5507),tensor(0.5870)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,tensor(137.),tensor(-1.),tensor(3.),tensor(0.),tensor(0.2026),tensor(0.0033),tensor(0.3301),tensor(0.0131),tensor(0.4542),tensor(0.0294),...,tensor(0.4510),tensor(0.6634),tensor(0.5327),tensor(0.6928),tensor(0.4510),tensor(0.7059),tensor(0.4020),tensor(0.7092),tensor(0.3529),tensor(0.6993)
803,tensor(137.),tensor(-1.),tensor(3.),tensor(0.),tensor(0.2824),tensor(0.0231),tensor(0.4179),tensor(0.0461),tensor(0.5504),tensor(0.0807),...,tensor(0.5591),tensor(0.6340),tensor(0.6686),tensor(0.6513),tensor(0.5648),tensor(0.6974),tensor(0.5043),tensor(0.7089),tensor(0.4467),tensor(0.7089)
1316,tensor(137.),tensor(-1.),tensor(5.),tensor(0.),tensor(0.1321),tensor(0.0038),tensor(0.2717),tensor(0.0113),tensor(0.4113),tensor(0.0302),...,tensor(0.5283),tensor(0.6679),tensor(0.6226),tensor(0.6868),tensor(0.5283),tensor(0.6906),tensor(0.4755),tensor(0.6943),tensor(0.4264),tensor(0.6792)
1488,tensor(137.),tensor(-1.),tensor(6.),tensor(0.),tensor(0.1667),tensor(0.0068),tensor(0.3061),tensor(0.0272),tensor(0.4388),tensor(0.0544),...,tensor(0.5918),tensor(0.6565),tensor(0.6701),tensor(0.6565),tensor(0.5918),tensor(0.6463),tensor(0.5408),tensor(0.6531),tensor(0.4830),tensor(0.6463)


In [36]:
y_train['label'].value_counts()

0.0    1432
1.0      98
Name: label, dtype: int64

In [20]:
df_y['label'].value_counts()

0.0    2161
1.0     728
Name: label, dtype: int64

In [18]:
X = torch.from_numpy(np.load('data/stratify_people/samples.npy'))
y = torch.from_numpy(np.load('data/stratify_people/labels.npy'))

X_train2, X_test2, y_train2, y_test2 = train_test_split(df_X, df_y, test_size=0.10, stratify=[y, pd.DataFrame(original_df_X['subject'])])

C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:673: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:673: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, orde

AttributeError: 'bool' object has no attribute 'any'